## Import des librairies

In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from P7_functions import dataset_overview
from P7_functions import NaN_overview
from P7_functions import feature_distribution

In [3]:
from Preprocess_Dataframe_V2 import downcast_numerical_column_V2

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
import imblearn
from imblearn.over_sampling import SMOTE

# 1. Ouverture du jeu de données

In [7]:
from IPython.display import display, HTML
plt.style.use('seaborn')

warnings.simplefilter(category=FutureWarning, action='ignore')
warnings.simplefilter(category=UserWarning, action='ignore')

# pour gérer l'affichage des dataframes pandas
pd.options.display.max_rows = 100
pd.options.display.max_columns = 200
pd.set_option("colheader_justify", "left")

# pour gérer l'affichage des arrays numpy
# 230 sur config bureau, 165 sur config portable
np.set_printoptions(linewidth=165)

# pour afficher les gros fichiers sur toute la largeur de l'écran
HTML("<style>.container { width:95% }</style>")

On conserve la notation df pour simplifier la lecture du Notebook. Attention, le fichier source est <b>df_train_test_outliers_processed.csv</b>!

In [8]:
df = pd.read_csv('df_train_test_outliers_processed.csv', compression = 'gzip')

In [9]:
df.head()

Unnamed: 0  AMT_ANNUITY  AMT_CREDIT  AMT_GOODS_PRICE  AMT_INCOME_TOTAL  \
0  0           24700.5       406597.5    351000.0        202500.0           
1  1           35698.5      1293502.5   1129500.0        270000.0           
2  2            6750.0       135000.0    135000.0         67500.0           
3  3           29686.5       312682.5    297000.0        135000.0           
4  4           21865.5       513000.0    513000.0        121500.0           

   AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_HOUR  \
0  0.0                        0.0                          
1  0.0                        0.0                          
2  0.0                        0.0                          
3  0.0                        0.0                          
4  0.0                        0.0                          

   AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
0  0.0                        0.0                         
1  0.0                        0.0                         
2  0.0                        0.0                         
3  0.0                        0.0                         
4  0.0                        0.0                         

   AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_YEAR  \
0  0.0                         1.0                          
1  0.0                         0.0                          
2  0.0                         0.0                          
3  0.0                         1.0                          
4  0.0                         0.0                          

   ANNUITY_INCOME_PERC  APPROVED_AMT_ANNUITY_MAX  APPROVED_AMT_ANNUITY_MEAN  \
0  0.121978              9251.775                  9251.775                   
1  0.132217             98356.995                 56553.990                   
2  0.100000              5357.250                  5357.250                   
3  0.219900             39954.510                 21842.190                   
4  0.179963             22678.785                 12278.805                   

   APPROVED_AMT_ANNUITY_MIN  APPROVED_AMT_APPLICATION_MAX  \
0  9251.775                  179055.0                       
1  6737.310                  900000.0                       
2  5357.250                   24282.0                       
3  2482.920                  675000.0                       
4  1834.290                  247500.0                       

   APPROVED_AMT_APPLICATION_MEAN  APPROVED_AMT_APPLICATION_MIN  \
0  179055.000                     179055.00                      
1  435436.500                      68809.50                      
2   24282.000                      24282.00                      
3  352265.868                      26912.34                      
4  150530.250                      17176.50                      

   APPROVED_AMT_CREDIT_MAX  APPROVED_AMT_CREDIT_MEAN  APPROVED_AMT_CREDIT_MIN  \
0   179055.0                179055.00                 179055.0                  
1  1035882.0                484191.00                  68053.5                  
2    20106.0                 20106.00                  20106.0                  
3   675000.0                343728.90                  24219.0                  
4   284400.0                166638.75                  14616.0                  

   APPROVED_AMT_DOWN_PAYMENT_MAX  APPROVED_AMT_DOWN_PAYMENT_MEAN  \
0      0.0                            0.00                         
1   6885.0                         3442.50                         
2   4860.0                         4860.00                         
3  66987.0                        34840.17                         
4   3676.5                         3390.75                         

   APPROVED_AMT_DOWN_PAYMENT_MIN  APPROVED_AMT_GOODS_PRICE_MAX  \
0     0.00                        179055.0                       
1     0.00                        900000.0                       
2  4860.00                         24282.0                       
3  2693.34                        675000.0                

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307471 entries, 0 to 307470
Columns: 195 entries, Unnamed: 0 to index
dtypes: float64(138), int64(57)
memory usage: 457.4 MB


In [11]:
df.drop('Unnamed: 0', axis=1, inplace = True)

In [12]:
df.info(memory_usage = 'deep', verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307471 entries, 0 to 307470
Data columns (total 194 columns):
 #    Column                                 Dtype  
---   ------                                 -----  
 0    AMT_ANNUITY                            float64
 1    AMT_CREDIT                             float64
 2    AMT_GOODS_PRICE                        float64
 3    AMT_INCOME_TOTAL                       float64
 4    AMT_REQ_CREDIT_BUREAU_DAY              float64
 5    AMT_REQ_CREDIT_BUREAU_HOUR             float64
 6    AMT_REQ_CREDIT_BUREAU_MON              float64
 7    AMT_REQ_CREDIT_BUREAU_QRT              float64
 8    AMT_REQ_CREDIT_BUREAU_WEEK             float64
 9    AMT_REQ_CREDIT_BUREAU_YEAR             float64
 10   ANNUITY_INCOME_PERC                    float64
 11   APPROVED_AMT_ANNUITY_MAX               float64
 12   APPROVED_AMT_ANNUITY_MEAN              float64
 13   APPROVED_AMT_ANNUITY_MIN               float64
 14   APPROVED_AMT_APPLICATION_MAX      

In [13]:
df = df.reindex(sorted(df.columns), axis=1)
df.head()

AMT_ANNUITY  AMT_CREDIT  AMT_GOODS_PRICE  AMT_INCOME_TOTAL  \
0  24700.5       406597.5    351000.0        202500.0           
1  35698.5      1293502.5   1129500.0        270000.0           
2   6750.0       135000.0    135000.0         67500.0           
3  29686.5       312682.5    297000.0        135000.0           
4  21865.5       513000.0    513000.0        121500.0           

   AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_HOUR  \
0  0.0                        0.0                          
1  0.0                        0.0                          
2  0.0                        0.0                          
3  0.0                        0.0                          
4  0.0                        0.0                          

   AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
0  0.0                        0.0                         
1  0.0                        0.0                         
2  0.0                        0.0                         
3  0.0                        0.0                         
4  0.0                        0.0                         

   AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_YEAR  \
0  0.0                         1.0                          
1  0.0                         0.0                          
2  0.0                         0.0                          
3  0.0                         1.0                          
4  0.0                         0.0                          

   ANNUITY_INCOME_PERC  APPROVED_AMT_ANNUITY_MAX  APPROVED_AMT_ANNUITY_MEAN  \
0  0.121978              9251.775                  9251.775                   
1  0.132217             98356.995                 56553.990                   
2  0.100000              5357.250                  5357.250                   
3  0.219900             39954.510                 21842.190                   
4  0.179963             22678.785                 12278.805                   

   APPROVED_AMT_ANNUITY_MIN  APPROVED_AMT_APPLICATION_MAX  \
0  9251.775                  179055.0                       
1  6737.310                  900000.0                       
2  5357.250                   24282.0                       
3  2482.920                  675000.0                       
4  1834.290                  247500.0                       

   APPROVED_AMT_APPLICATION_MEAN  APPROVED_AMT_APPLICATION_MIN  \
0  179055.000                     179055.00                      
1  435436.500                      68809.50                      
2   24282.000                      24282.00                      
3  352265.868                      26912.34                      
4  150530.250                      17176.50                      

   APPROVED_AMT_CREDIT_MAX  APPROVED_AMT_CREDIT_MEAN  APPROVED_AMT_CREDIT_MIN  \
0   179055.0                179055.00                 179055.0                  
1  1035882.0                484191.00                  68053.5                  
2    20106.0                 20106.00                  20106.0                  
3   675000.0                343728.90                  24219.0                  
4   284400.0                166638.75                  14616.0                  

   APPROVED_AMT_DOWN_PAYMENT_MAX  APPROVED_AMT_DOWN_PAYMENT_MEAN  \
0      0.0                            0.00                         
1   6885.0                         3442.50                         
2   4860.0                         4860.00                         
3  66987.0                        34840.17                         
4   3676.5                         3390.75                         

   APPROVED_AMT_DOWN_PAYMENT_MIN  APPROVED_AMT_GOODS_PRICE_MAX  \
0     0.00                        179055.0                       
1     0.00                        900000.0                       
2  4860.00                         24282.0                       
3  2693.34                        675000.0                       
4  3105.00                        247500.0                      

In [14]:
df.describe()

AMT_ANNUITY    AMT_CREDIT    AMT_GOODS_PRICE  AMT_INCOME_TOTAL  \
count  307471.000000  3.074710e+05  3.074710e+05     3.074710e+05       
mean    27107.061951  5.990002e+05  5.382900e+05     1.684104e+05       
std     14486.892686  4.024317e+05  3.692183e+05     1.087871e+05       
min      1615.500000  4.500000e+04  4.050000e+04     2.565000e+04       
25%     16524.000000  2.700000e+05  2.385000e+05     1.125000e+05       
50%     24903.000000  5.135310e+05  4.500000e+05     1.469970e+05       
75%     34596.000000  8.086500e+05  6.795000e+05     2.025000e+05       
max    230161.500000  4.050000e+06  4.050000e+06     1.800009e+07       

       AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_HOUR  \
count  307471.000000              307471.000000                
mean        0.006056                   0.005539                
std         0.103044                   0.078019                
min         0.000000                   0.000000                
25%         0.000000                   0.000000                
50%         0.000000                   0.000000                
75%         0.000000                   0.000000                
max         9.000000                   4.000000                

       AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
count  307471.000000              307471.000000               
mean        0.231303                   0.228731               
std         0.856856                   0.575635               
min         0.000000                   0.000000               
25%         0.000000                   0.000000               
50%         0.000000                   0.000000               
75%         0.000000                   0.000000               
max        27.000000                   8.000000               

       AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_YEAR  \
count  307471.000000               307471.000000                
mean        0.029723                    1.778447                
std         0.190732                    1.765573                
min         0.000000                    0.000000                
25%         0.000000                    1.000000                
50%         0.000000                    1.000000                
75%         0.000000                    3.000000                
max         8.000000                   25.000000                

       ANNUITY_INCOME_PERC  APPROVED_AMT_ANNUITY_MAX  \
count  307471.000000        307471.000000              
mean        0.180923         21322.192068              
std         0.094520         17242.272154              
min         0.003333             0.000000              
25%         0.114785          9934.245000              
50%         0.162833         16143.525000              
75%         0.229050         27084.240000              
max         1.570600        393868.665000              

       APPROVED_AMT_ANNUITY_MEAN  APPROVED_AMT_ANNUITY_MIN  \
count  307471.000000              307471.000000              
mean    13779.353469                8127.257008              
std      9577.609383                7710.911304              
min         0.000000                   0.000000              
25%      7703.880000                3857.512500              
50%     11373.480000                5998.680000              
75%     16941.630000                9590.400000              
max    300425.445000              300425.445000              

       APPROVED_AMT_APPLICATION_MAX  APPROVED_AMT_APPLICATION_MEAN  \
count  3.074710e+05                  3.074710e+05                    
mean   3.028417e+05                  1.628214e+05                    
std    3.422219e+05                  1.591891e+05                    
min    0.000000e+00                  0.000000e+00                    
25%    9.084600e+04                  6.839100e+04                    
50%    1.691865e+05                  1.132268e+05                    
75%    3.780000e+05                  1.978875e+05               

In [15]:
NaN_overview(df)

Total number of columns:  194
Number of columns with NaN:  0
AMT_ANNUITY                              0.0
AMT_CREDIT                               0.0
AMT_GOODS_PRICE                          0.0
AMT_INCOME_TOTAL                         0.0
AMT_REQ_CREDIT_BUREAU_DAY                0.0
AMT_REQ_CREDIT_BUREAU_HOUR               0.0
AMT_REQ_CREDIT_BUREAU_MON                0.0
AMT_REQ_CREDIT_BUREAU_QRT                0.0
AMT_REQ_CREDIT_BUREAU_WEEK               0.0
AMT_REQ_CREDIT_BUREAU_YEAR               0.0
ANNUITY_INCOME_PERC                      0.0
APPROVED_AMT_ANNUITY_MAX                 0.0
APPROVED_AMT_ANNUITY_MEAN                0.0
APPROVED_AMT_ANNUITY_MIN                 0.0
APPROVED_AMT_APPLICATION_MAX             0.0
APPROVED_AMT_APPLICATION_MEAN            0.0
APPROVED_AMT_APPLICATION_MIN             0.0
APPROVED_AMT_CREDIT_MAX                  0.0
APPROVED_AMT_CREDIT_MEAN                 0.0
APPROVED_AMT_CREDIT_MIN                  0.0
APPROVED_AMT_DOWN_PAYMENT_MAX          

# 2 . Séparation entre le jeu d'entraînement/test et le jeu de validation

- df_train_test sera utilisé pour l'entraînement et le cross validation.<br>
- df_valid_tt ne sera pas du tout vu par l'entraînement ni la cross validation et servira uniquement pour vérifier la performance.

In [16]:
train_test, valid_tt = train_test_split(df, test_size=1000, stratify=df['TARGET'], random_state=42)

In [17]:
df_valid_tt = pd.DataFrame(valid_tt)

In [18]:
df_train_test = pd.DataFrame(train_test)

In [19]:
df_valid_tt.to_csv('df_valid_tt.csv', compression = 'gzip')

In [20]:
df_train_test.to_csv('df_train_test.csv', compression = 'gzip')

In [21]:
df_valid_tt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 144164 to 298448
Columns: 194 entries, AMT_ANNUITY to index
dtypes: float64(138), int64(56)
memory usage: 1.5 MB


In [22]:
df_train_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306471 entries, 299904 to 279635
Columns: 194 entries, AMT_ANNUITY to index
dtypes: float64(138), int64(56)
memory usage: 455.9 MB


# 3. Préparation des jeux de données d'entraînement : X (features) et y (target)

## 3.1. Création de X et y

In [23]:
X = df_train_test.drop(columns='TARGET')

In [24]:
X.shape

(306471, 193)

In [25]:
y = df_train_test[['TARGET']]

In [26]:
y.shape

(306471, 1)

In [27]:
pd.set_option('display.max_columns', None)
X.head()

AMT_ANNUITY  AMT_CREDIT  AMT_GOODS_PRICE  AMT_INCOME_TOTAL  \
299904  19953.0      247275.0    225000.0         135000.0           
249970  15750.0      315000.0    315000.0          99000.0           
149060  14233.5      276277.5    238500.0          60750.0           
278013  41787.0      781920.0    675000.0         315000.0           
221527   9000.0      180000.0    180000.0         112500.0           

        AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_HOUR  \
299904  0.0                        0.0                          
249970  0.0                        0.0                          
149060  0.0                        0.0                          
278013  0.0                        0.0                          
221527  0.0                        0.0                          

        AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
299904  1.0                        0.0                         
249970  0.0                        1.0                         
149060  0.0                        0.0                         
278013  1.0                        0.0                         
221527  0.0                        0.0                         

        AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_YEAR  \
299904  0.0                         0.0                          
249970  0.0                         0.0                          
149060  0.0                         2.0                          
278013  0.0                         3.0                          
221527  0.0                         1.0                          

        ANNUITY_INCOME_PERC  APPROVED_AMT_ANNUITY_MAX  \
299904  0.147800              7368.21                   
249970  0.159091             29829.96                   
149060  0.234296              6609.06                   
278013  0.132657             22500.00                   
221527  0.080000             47173.68                   

        APPROVED_AMT_ANNUITY_MEAN  APPROVED_AMT_ANNUITY_MIN  \
299904   7368.2100                  7368.210                  
249970  12316.2975                  2451.015                  
149060   5150.5650                  2250.000                  
278013  19246.9050                 15993.810                  
221527  28691.7975                 10209.915                  

        APPROVED_AMT_APPLICATION_MAX  APPROVED_AMT_APPLICATION_MEAN  \
299904  160659.0                      160659.00                       
249970  495000.0                      199712.25                       
149060   49455.0                       43483.50                       
278013  129456.0                       64728.00                       
221527  479700.0                      295177.50                       

        APPROVED_AMT_APPLICATION_MIN  APPROVED_AMT_CREDIT_MAX  \
299904  160659.0                      160659.0                  
249970   31495.5                      534204.0                  
149060   35995.5                       54351.0                  
278013       0.0                      450000.0                  
221527  110655.0                      479700.0                  

        APPROVED_AMT_CREDIT_MEAN  APPROVED_AMT_CREDIT_MIN  \
299904  160659.000                160659.0                  
249970  218458.125                 28332.0                  
149060   47662.500                 43636.5                  
278013  296136.000                142272.0                  
221527  289644.750                 99589.5                  

        APPROVED_AMT_DOWN_PAYMENT_MAX  APPROVED_AMT_DOWN_PAYMENT_MEAN  \
299904      0.0                           0.00                          
249970   4500.0                        1500.00                          
149060      0.0                           0.00                          
278013      0.0                           0.00                          
221527  11065.5                        5532.75                          

        APPROVED_AMT_DOWN_PAYMENT_MIN  APPROVED_AMT_GOODS_PRICE_MAX 

In [28]:
y.head()

TARGET
299904  1.0   
249970  0.0   
149060  0.0   
278013  0.0   
221527  0.0

In [29]:
X.to_csv('X.csv', compression = 'gzip')

In [30]:
y.to_csv('y.csv', compression = 'gzip')